In [1]:
#Library imports
import requests
from IPython.display import JSON 
import pandas as pd
import numpy as np
import os

In [2]:
df_marseille_stations = pd.read_csv("../data/df_marseille_stations.csv")
marseille_stations = df_marseille_stations.to_dict(orient="records")


In [4]:
def fsq_stations_scrape():
    fsq_stations_scrape_array = []
    
    for stations in marseille_stations:
        name = stations["name"]
        free_bikes = stations["free_bikes"]
        Latitude = stations["Latitude"]
        Longitude = stations["Longitude"]
        dict_of_each_poi = {"name": name, "free_bikes" : free_bikes, "Latitude": Latitude, "Longitude": Longitude}
        fsq_stations_scrape_array.append(dict_of_each_poi)
        
    return fsq_stations_scrape_array

parsed_stations = fsq_stations_scrape()

for station in parsed_stations[:10]:
    print(station)

{'name': 'Cours Jean Ballard', 'free_bikes': 0, 'Latitude': 43.29300892, 'Longitude': 5.374584208}
{'name': 'Allées Gambetta', 'free_bikes': 0, 'Latitude': 43.29902797, 'Longitude': 5.381509226}
{'name': 'Réformés Canebière', 'free_bikes': 2, 'Latitude': 43.29971664630074, 'Longitude': 5.384845163546515}
{'name': 'Coutellerie', 'free_bikes': 1, 'Latitude': 43.29679958, 'Longitude': 5.372909688}
{'name': 'Canebiere - Beauvau', 'free_bikes': 0, 'Latitude': 43.29527892, 'Longitude': 5.375048465}
{'name': 'Colbert Ste Barbe', 'free_bikes': 3, 'Latitude': 43.299242645590766, 'Longitude': 5.374807305735999}
{'name': 'Centre Bourse', 'free_bikes': 1, 'Latitude': 43.29693502454282, 'Longitude': 5.376544207099414}
{'name': 'Saint-Ferréol - Davso', 'free_bikes': 1, 'Latitude': 43.29360042, 'Longitude': 5.378349882}
{'name': 'Flammarion - Isoard', 'free_bikes': 1, 'Latitude': 43.3042781513621, 'Longitude': 5.388867929313619}
{'name': 'Longchamp - Grobet', 'free_bikes': 1, 'Latitude': 43.302992339

# Foursquare

*This API broke on me mid analysis and was loading empty rows. I played around with the syntax, but was unable to get it to function again, even though it appeared to function initially. so, for the sake of completing the assignment, I have chosen to omit the foursquare data completely.*

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [5]:
yelp_api_key = ''

def yelp_radius(parsed_stations, radius=1000):
    yelp_responses = []  
    
    for station in parsed_stations:
        latitude = station["Latitude"]
        longitude = station["Longitude"]
        
        yelp_url = f"https://api.yelp.com/v3/businesses/search?latitude={latitude}&longitude={longitude}&radius=1000&categories=&sort_by=best_match&limit=10"


        headers = {
            'Authorization': f'Bearer {yelp_api_key}'
        }

        response = requests.get(yelp_url, headers=headers)

        if response.status_code == 200:
            yelp_response = response.json()
            yelp_responses.append(yelp_response)
        else:
            print(f"API request failed with status code {response.status_code}")

    return yelp_responses

yelp_responses = yelp_radius(parsed_stations)


for response in yelp_responses[:10]:
    if "businesses" in response:
        businesses = response["businesses"]
        for business in businesses:
            print("Business Name:", business.get("name", "N/A"))
            print("Rating:", business.get("rating", "N/A"))
            print("Review Count:", business.get("review_count", "N/A"))
            print("------------")


Business Name: Shanghai Kitchen
Rating: 5.0
Review Count: 10
------------
Business Name: Le Bon Burger
Rating: 4.5
Review Count: 36
------------
Business Name: Le Petit Flore
Rating: 5.0
Review Count: 24
------------
Business Name: Grumpy Cakes
Rating: 4.5
Review Count: 35
------------
Business Name: Le Grand Latin
Rating: 4.0
Review Count: 179
------------
Business Name: La Table du Fort
Rating: 4.5
Review Count: 58
------------
Business Name: Le Bistrot à Vin
Rating: 4.0
Review Count: 76
------------
Business Name: Green Bear Coffee
Rating: 4.5
Review Count: 46
------------
Business Name: Les Panisses
Rating: 4.0
Review Count: 39
------------
Business Name: Le Bouchon Provençal
Rating: 4.0
Review Count: 39
------------
Business Name: Le Cercle Rouge
Rating: 4.5
Review Count: 65
------------
Business Name: Sauveur
Rating: 4.5
Review Count: 132
------------
Business Name: La Manne
Rating: 4.5
Review Count: 12
------------
Business Name: Le Grain de Sable
Rating: 4.5
Review Count: 47
--

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [86]:
def yelp_data(yelp_responses):
    yelp_data_array = []
    for poi in yelp_responses:
        businesses = poi.get("businesses", [])  
        for business in businesses:
            id = business["id"]
            name = business["name"]
            categories = business.get("categories", [])  
            distance = int(business["distance"])
            address = business["location"]["address1"]
            latitude = business["coordinates"]["latitude"]  
            longitude = business["coordinates"]["longitude"]  
            review_count = int(business["review_count"])
            rating = float(business["rating"])
            dict_of_each_poi = {"Id": id, "Name": name, "Category": categories, "Distance": distance, "address": address, "Latitude": latitude,
                                 "Longitude": longitude, "review_count": review_count, "rating": rating}
            yelp_data_array.append(dict_of_each_poi)
    return yelp_data_array


 

In [87]:
yelp_restaurant_data = yelp_data(yelp_responses)

In [88]:
yelp_restaurant_data = [item for sublist in yelp_restaurant_data for item in (sublist if isinstance(sublist, list) else [sublist])]

In [89]:

df_yelp_restaurant_data = pd.DataFrame(yelp_restaurant_data, columns=["Id", "Name", "Category", "Distance", "Address", "Latitude", "Longitude", 
                                                                      "Review_count", "Rating"])


In [90]:
columns_to_keep = ["Id","Name", "Category", "Distance", "Address", "Latitude", "Longitude", "Review_count", "Rating"]
df_yelp_restaurant_data_revised = df_yelp_restaurant_data[columns_to_keep]


print(df_yelp_restaurant_data_revised[:5])

                       Id              Name  \
0  Z_ezFqDSgos1VLmHdePBqg  Shanghai Kitchen   
1  gMbdTQCBPjp7mDfWJehHmQ     Le Bon Burger   
2  uLqZhed6FOI-Yxmdi-W2tA    Le Petit Flore   
3  VO2Gk3YV0vMLEkc3jg0_BA      Grumpy Cakes   
4  Jpa7n24xU5h_UTOkbQGyJg    Le Grand Latin   

                                            Category  Distance  \
0  [{'alias': 'shanghainese', 'title': 'Shanghain...         6   
1  [{'alias': 'burgers', 'title': 'Burgers'}, {'a...       228   
2  [{'alias': 'mediterranean', 'title': 'Mediterr...        78   
3  [{'alias': 'tea', 'title': 'Tea Rooms'}, {'ali...        82   
4         [{'alias': 'italian', 'title': 'Italian'}]       104   

                 Address   Latitude  Longitude  Review_count  Rating  
0  14 cours Jean Ballard  43.292984   5.374505            10     5.0  
1    24 place aux Huiles  43.292656   5.371802            36     4.5  
2       14 rue Corneille  43.293307   5.375469            24     5.0  
3       12 rue Corneille  43.293379 

In [91]:
df_yelp_restaurant_data_revised.shape 

(1846, 9)

In [92]:
df_yelp_restaurant_data_revised.columns

Index(['Id', 'Name', 'Category', 'Distance', 'Address', 'Latitude',
       'Longitude', 'Review_count', 'Rating'],
      dtype='object')

In [93]:
df_yelp_restaurant_data_revised.dtypes

Id               object
Name             object
Category         object
Distance          int64
Address          object
Latitude        float64
Longitude       float64
Review_count      int64
Rating          float64
dtype: object

In [95]:
df_yelp_restaurant_data_revised.duplicated(subset=["Id"]).sum()

1345

In [96]:
df_yelp_restaurant_data_revised.loc[df_yelp_restaurant_data_revised.duplicated(subset=["Id", "Name"])][:5]

,Id,Name,Category,Distance,Address,Latitude,Longitude,Review_count,Rating
20,wwcX2d_9rQmz9OzUQEVFHw,Le Cercle Rouge,"[{'alias': 'tapas', 'title': 'Tapas Bars'}, {'...",273,41 rue Adolphe Thiers,43.297290,5.385460,65,4.5
22,tpeRKLNE9Vn5EtihtKEMqQ,La Boîte à Sardine,"[{'alias': 'seafood', 'title': 'Seafood'}]",122,2 boulevard de la Libération,43.299250,5.386180,39,4.0
23,ZAHTxYqDTbORoZ6I7G9aGw,La Manne,"[{'alias': 'french', 'title': 'French'}, {'ali...",249,18 boulevard de la Liberté,43.300773,5.382120,12,4.5
25,EfMabFwgjmWoUibT1DMMzw,Sauveur,"[{'alias': 'pizza', 'title': 'Pizza'}]",655,10 rue d'Aubagne,43.295589,5.379063,132,4.5
26,KgYoPVXM50ZRhWeDl4mlMA,Le Corto,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",886,24 place Notre-Dame du Mont,43.291757,5.384225,65,4.5


In [97]:
df_yelp_restaurant_data_revised = df_yelp_restaurant_data_revised.loc[~df_yelp_restaurant_data_revised.duplicated(subset=["Id", "Name"])] \
.reset_index(drop=True)

In [98]:
df_yelp_restaurant_data_revised.shape

(501, 9)

In [100]:
df_yelp_restaurant_data_revised[:10]

,Id,Name,Category,Distance,Address,Latitude,Longitude,Review_count,Rating
0,Z_ezFqDSgos1VLmHdePBqg,Shanghai Kitchen,"[{'alias': 'shanghainese', 'title': 'Shanghain...",6,14 cours Jean Ballard,43.292984,5.374505,10,5.0
1,gMbdTQCBPjp7mDfWJehHmQ,Le Bon Burger,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",228,24 place aux Huiles,43.292656,5.371802,36,4.5
2,uLqZhed6FOI-Yxmdi-W2tA,Le Petit Flore,"[{'alias': 'mediterranean', 'title': 'Mediterr...",78,14 rue Corneille,43.293307,5.375469,24,5.0
3,VO2Gk3YV0vMLEkc3jg0_BA,Grumpy Cakes,"[{'alias': 'tea', 'title': 'Tea Rooms'}, {'ali...",82,12 rue Corneille,43.293379,5.375462,35,4.5
4,Jpa7n24xU5h_UTOkbQGyJg,Le Grand Latin,"[{'alias': 'italian', 'title': 'Italian'}]",104,27 rue Sainte,43.292530,5.375580,179,4.0
5,Pvzn2DLyJgUWwU6X54zHZw,La Table du Fort,"[{'alias': 'french', 'title': 'French'}]",264,8 rue Fort Notre Dame,43.292740,5.371340,58,4.5
6,cXLbCKm1pyJqEa9BdsyCyw,Le Bistrot à Vin,"[{'alias': 'bistros', 'title': 'Bistros'}, {'a...",127,17 rue Sainte,43.292630,5.376010,76,4.0
7,yzY4RE7EAcjTOg3KBMi2MQ,Green Bear Coffee,"[{'alias': 'gluten_free', 'title': 'Gluten-Fre...",70,17 rue Glandèves,43.293440,5.375170,46,4.5
8,OXU6KQgkDc7aM9UFDYq1nQ,Les Panisses,"[{'alias': 'french', 'title': 'French'}, {'ali...",101,23 rue Sainte,43.292610,5.375710,39,4.0
9,ApiCSbuCSLk61vDLr2DV-g,Le Bouchon Provençal,"[{'alias': 'restaurants', 'title': 'Restaurant...",227,6 place aux Huiles,43.292870,5.371780,39,4.0


In [101]:
df_yelp_restaurant_data_revised["Category"] = df_yelp_restaurant_data_revised["Category"].apply(lambda x: [item['title'] for item in x] if isinstance(x, list) else [])

In [102]:
df_yelp_restaurant_data_revised[:10]

,Id,Name,Category,Distance,Address,Latitude,Longitude,Review_count,Rating
0,Z_ezFqDSgos1VLmHdePBqg,Shanghai Kitchen,[Shanghainese],6,14 cours Jean Ballard,43.292984,5.374505,10,5.0
1,gMbdTQCBPjp7mDfWJehHmQ,Le Bon Burger,"[Burgers, American]",228,24 place aux Huiles,43.292656,5.371802,36,4.5
2,uLqZhed6FOI-Yxmdi-W2tA,Le Petit Flore,"[Mediterranean, French]",78,14 rue Corneille,43.293307,5.375469,24,5.0
3,VO2Gk3YV0vMLEkc3jg0_BA,Grumpy Cakes,"[Tea Rooms, Desserts, Salad]",82,12 rue Corneille,43.293379,5.375462,35,4.5
4,Jpa7n24xU5h_UTOkbQGyJg,Le Grand Latin,[Italian],104,27 rue Sainte,43.292530,5.375580,179,4.0
5,Pvzn2DLyJgUWwU6X54zHZw,La Table du Fort,[French],264,8 rue Fort Notre Dame,43.292740,5.371340,58,4.5
6,cXLbCKm1pyJqEa9BdsyCyw,Le Bistrot à Vin,"[Bistros, Wine Bars]",127,17 rue Sainte,43.292630,5.376010,76,4.0
7,yzY4RE7EAcjTOg3KBMi2MQ,Green Bear Coffee,"[Gluten-Free, Vegetarian, Cafes]",70,17 rue Glandèves,43.293440,5.375170,46,4.5
8,OXU6KQgkDc7aM9UFDYq1nQ,Les Panisses,"[French, Brasseries]",101,23 rue Sainte,43.292610,5.375710,39,4.0
9,ApiCSbuCSLk61vDLr2DV-g,Le Bouchon Provençal,[Restaurants],227,6 place aux Huiles,43.292870,5.371780,39,4.0


In [107]:
df_yelp_restaurant_data_revised.isna().sum()

Id              0
Name            0
Category        0
Distance        0
Address         2
Latitude        0
Longitude       0
Review_count    0
Rating          0
dtype: int64

In [109]:
missing_values_rows = df_yelp_restaurant_data_revised[df_yelp_restaurant_data_revised.isna().any(axis=1)]

print(missing_values_rows)

                         Id                        Name              Category  \
268  M3vcgKudCOOepx0CX0EJ7w           fast food chinois  [Fast Food, Chinese]   
351  zpuQFsJMcFCQ_3_ZZecqbw  Les Calanques de Marseille        [Local Flavor]   

     Distance Address   Latitude  Longitude  Review_count  Rating  
268       218    None  43.298544   5.433746             1     4.0  
351       892    None  43.226650   5.368700            43     4.5  


Put your parsed results into a DataFrame

In [116]:
df_yelp_restaurant_data = pd.DataFrame(df_yelp_restaurant_data_revised, columns=["Name", "Category", "Distance", "Address", "Latitude", "Longitude", 
                                                                      "Review_count", "Rating"])


In [117]:
df_yelp_restaurant_data[:5]

,Name,Category,Distance,Address,Latitude,Longitude,Review_count,Rating
0,Shanghai Kitchen,[Shanghainese],6,14 cours Jean Ballard,43.292984,5.374505,10,5.0
1,Le Bon Burger,"[Burgers, American]",228,24 place aux Huiles,43.292656,5.371802,36,4.5
2,Le Petit Flore,"[Mediterranean, French]",78,14 rue Corneille,43.293307,5.375469,24,5.0
3,Grumpy Cakes,"[Tea Rooms, Desserts, Salad]",82,12 rue Corneille,43.293379,5.375462,35,4.5
4,Le Grand Latin,[Italian],104,27 rue Sainte,43.292530,5.375580,179,4.0


In [118]:
len(df_yelp_restaurant_data)

501

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

*The yelp data was more complete, and functional. Th Foursquare data was omitted from the analytic process.*

Get the top 10 restaurants according to their rating

In [119]:
print(df_yelp_restaurant_data.columns)



Index(['Name', 'Category', 'Distance', 'Address', 'Latitude', 'Longitude',
       'Review_count', 'Rating'],
      dtype='object')


In [123]:
top_10_restaurants = df_yelp_restaurant_data.sort_values(by=["Rating","Review_count"], ascending=False).head(10)

top_10_restaurants

,Name,Category,Distance,Address,Latitude,Longitude,Review_count,Rating
2,Le Petit Flore,"[Mediterranean, French]",78,14 rue Corneille,43.293307,5.375469,24,5.0
169,Le Patio du Prado,[French],735,9 Rue Borde,43.278260,5.388980,18,5.0
122,Le Citronnier,"[French, International, South African]",440,2 rue Elemir Bourges,43.298800,5.401050,15,5.0
0,Shanghai Kitchen,[Shanghainese],6,14 cours Jean Ballard,43.292984,5.374505,10,5.0
190,L'Horloge du Cap Est,"[French, Breakfast & Brunch, Cafes]",585,10 boulevard Fernand Bonnefoy,43.281925,5.400787,9,5.0
317,Bière de la Plaine,"[Beer Bar, Breweries]",281,16 rue Saint Pierre,43.293670,5.387620,9,5.0
107,Pâtisserie B et F Giner La Gourmandine,[Bakeries],66,8 Place Bernard Cadenat,43.311867,5.386445,6,5.0
94,Frac Paca,[Museums],32,20 bd de Dunkerque,43.305760,5.367530,5,5.0
435,Les Mains Libres,[Bakeries],44,117 bd Chave,43.295530,5.393900,4,5.0
380,Jean Sauveur,"[Pizza, Napoletana]",876,33 Boulevard de Saint-Loup,43.282006,5.429565,3,5.0


In [124]:
df_yelp_restaurant_data.to_csv("../data/df_yelp_data.csv", index=True)
df_yelp_restaurant_data_revised.to_csv("../data/df_yelp_restaurant_data_revised.csv", index=True)